In [1]:
import synapseclient
from synapseclient import Project, File, Folder
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import itertools
import pandas as pd
import numpy as np

In [2]:
syn = synapseclient.Synapse()
syn.login()

Welcome, Victor Baham!



In [21]:
divco_bio = pd.read_csv(syn.get('syn51757645').path)
rosmap_bio = pd.read_csv(syn.get('syn21323366').path)
mayo_bio = pd.read_csv(syn.get('syn20827192').path)
msbb_bio = pd.read_csv(syn.get('syn21893059').path)

#### Collapse tissue types into broader categories

In [33]:
tissue_to_category = {'occipital visual cortex': 'visual cortex', 
                     'parahippocampal gyrus': 'temporal cortex',
                     'posterior cingulate cortex': 'cingulate cortex',
                     'serum': 'blood',
                     'Head of caudate nucleus': 'caudate nucleus',
                     'caudate nucleus': 'caudate nucleus',
                     'cerebellar cortex': 'cerebellum',
                     'dorsolateral prefrontal cortex': 'prefrontal cortex',
                     'frontal pole': 'frontal cortex',
                     'inferior frontal gyrus': 'frontal cortex',
                     'superior temporal gyrus': 'temporal cortex',
                     'temporal pole': 'temporal cortex'}

In [22]:
rosmap_bio['specimenMetadataSource'] = 'ROSMAP'
mayo_bio['specimenMetadataSource'] = 'MayoRNAseq'
msbb_bio['specimenMetadataSource'] = 'MSBB'

In [38]:
mega_bio = pd.concat([divco_bio, rosmap_bio, mayo_bio, msbb_bio], ignore_index=True)
mega_bio['tissue'] = mega_bio['tissue'].apply(lambda x: tissue_to_category.get(x) if x 
                                              in tissue_to_category.keys() else x)
mega_bio

[WARNING] /Users/vbaham/opt/anaconda3/lib/python3.9/site-packages/IPython/lib/pretty.py:700: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)

[WARNING] /Users/vbaham/opt/anaconda3/lib/python3.9/site-packages/IPython/core/formatters.py:345: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()



,individualID,specimenID,dataGenerationSite,organ,tissue,BrodmannArea,isPostMortem,sampleStatus,nucleicAcidSource,assay,specimenMetadataSource,specimenIdSource,tissueWeight,tissueVolume,cellType,fastingState,samplingAge,samplingAgeUnits,visitNumber,exclude,excludeReason,samplingDate,Exclusion_Category
0,NYBB_17,NYBB_17WGS,NYGC,brain,prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,AMP-AD_DiverseCohorts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NYBB_147,NYBB_147WGS,NYGC,brain,prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,AMP-AD_DiverseCohorts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NYBB_148,NYBB_148WGS,NYGC,brain,prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,AMP-AD_DiverseCohorts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NYBB_153,NYBB_153WGS,NYGC,brain,prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,AMP-AD_DiverseCohorts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NYBB_76,NYBB_76WGS,NYGC,brain,prefrontal cortex,BA9,True,frozen,bulk cell,wholeGenomeSeq,AMP-AD_DiverseCohorts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample swap,NaN,NaN


#### Find number of distinct specimens for each assay type:

In [42]:
unique_specs_per_assay = mega_bio.groupby(['assay', 'specimenID'])['specimenID'].nunique()
unique_specs_per_assay_piv_1 = mega_bio.groupby(['assay', 'specimenID'],as_index = False).nunique().pivot('assay'
                    ,'specimenID').fillna(0)
unique_specs_per_assay_piv_2 = mega_bio.groupby(['specimenID', 'assay']).size().unstack('assay', fill_value=0.0)
unique_specs_per_assay_piv_2

[WARNING] /var/folders/lg/w3yckgy936lc0r6yw790hjs00000gr/T/ipykernel_2475/1207037233.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  unique_specs_per_assay_piv_1 = mega_bio.groupby(['assay', 'specimenID'],as_index = False).nunique().pivot('assay'



assay,10x multiome,ATACSeq,ATACseq,Biocrates Bile Acids,Biocrates p180,ChIPSeq,Metabolon,TMT quantitation,exomeSeq,label free mass spectrometry,methylationArray,mirnaArray,rnaArray,rnaSeq,scrnaSeq,snpArray,wholeGenomeSeq
specimenID,,,,,,,,,,,,,,,,,
01_120405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
02_120405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
03_120405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
04_120405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
05_120405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sd_b25.131N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sd_b25.132C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sd_b25.132N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Find number of specimens of each tissue type for each assay type:

In [40]:
unique_specs_per_assay_and_tissue = mega_bio.groupby(['assay', 'specimenID', 'tissue'])['specimenID'].count()
unique_specs_per_assay_and_tissue_piv = mega_bio.groupby(['assay', 'tissue'])['specimenID'].count().unstack('tissue', fill_value=0.0)
unique_specs_per_assay_and_tissue_piv

tissue,blood,caudate nucleus,cerebellum,cingulate cortex,frontal cortex,prefrontal cortex,temporal cortex,unspecified,visual cortex
assay,,,,,,,,,
10x multiome,0.0,128.0,0.0,0.0,0.0,135.0,136.0,0.0,0.0
ATACSeq,0.0,0.0,0.0,0.0,0.0,0.0,636.0,0.0,0.0
ATACseq,0.0,0.0,0.0,0.0,0.0,0.0,110.0,0.0,0.0
Biocrates Bile Acids,0.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,0.0
Biocrates p180,597.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,0.0
ChIPSeq,0.0,0.0,0.0,0.0,0.0,712.0,0.0,0.0,0.0
Metabolon,0.0,0.0,182.0,0.0,0.0,514.0,196.0,0.0,0.0
TMT quantitation,0.0,0.0,0.0,0.0,0.0,1505.0,470.0,0.0,0.0
exomeSeq,0.0,0.0,0.0,0.0,83.0,0.0,263.0,0.0,0.0


In [11]:
pd.set_option('display.max_columns', None)